In [4]:
import requests
from bs4 import BeautifulSoup
import json


In [5]:
def fetch_faculty_info(url):
  # Example URL - replace with the actual faculty page URL
  url = "https://engineering.buffalo.edu" + url
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  # Assuming soup is already defined with your HTML content
  sections = soup.find_all('a', class_="ada-hidden")
  faculty_info = {}
  for section in sections:
    # Dictionary to store faculty information
    if ('Bio' in section.get_text(strip=True)):
        response = requests.get(section['href'])
        soup1 = BeautifulSoup(response.text, 'html.parser')


        biography = {}
        # Extracting name and qualification
        name_element = soup1.find('h1')
        biography['name'] = name_element.get_text(strip=True).split(',')[0] if name_element else "Name Not Available"

        # Extracting photo URL
        photo_element = soup1.find('img', class_='img-140')
        biography['photo_url'] = photo_element['src'] if photo_element else "Photo URL Not Available"

        # Extracting department and school information
        dept_title_element = soup1.find('div', class_='profileinfo-dept-title')
        biography['dept_title'] = dept_title_element.get_text(strip=True) if dept_title_element else "Dept Title Not Available"

        dept_name_element = soup1.find('div', class_='profileinfo-dept-name')
        biography['dept_name'] = dept_name_element.get_text(strip=True) if dept_name_element else "Dept Name Not Available"

        school_name_element = soup1.find('div', class_='profileinfo-school-name')
        biography['school_name'] = school_name_element.get_text(strip=True) if school_name_element else "School Name Not Available"

        research_topics_element = soup1.find('div', class_='profileinfo-interest')
        if research_topics_element:
            research_topics_text = research_topics_element.get_text(strip=True)
            # Assuming the topics are separated by commas
            research_topics = [topic.strip() for topic in research_topics_text.replace('Research Topics', '').split(',')]
            biography['research_topics'] = research_topics
        else:
            biography['research_topics'] = []

        # Extracting contact information
        contact_info_section = soup1.find('div', class_='profileinfo-contact')
        if contact_info_section:
            address = contact_info_section.find('p').get_text(strip=True) if contact_info_section.find('p') else "Address Not Available"
            phone = contact_info_section.find(string=lambda x: "Phone:" in x)
            phone = phone.strip() if phone else "Phone Not Available"
            email = contact_info_section.find('a', class_='longtext')['href'] if contact_info_section.find('a', class_='longtext') and contact_info_section.find('a', class_='longtext')['href'] else "Email Not Available"

            biography['contact_info'] = {
                'address': address,
                'phone': phone,
                'email': email
            }
        else:
            biography['contact_info'] = "Contact Info Not Available"

        # Extracting related links
        related_links_section = soup1.find('div', id='profileinfo-links')
        if related_links_section:
            links = related_links_section.find_all('a')
            related_links = [{'text': link.get_text(strip=True), 'url': link['href']} for link in links if link and 'href' in link]
            biography['related_links'] = related_links
        else:
            biography['related_links'] = []

         # Assuming soup1 is already defined with your HTML content
        subsections = soup1.find_all('div', class_='title section')

        for subsection in subsections:
            # Extracting the title from the h3 tag
            title_element = subsection.find('h3')
            if not title_element:
                continue

            title = title_element.get_text(strip=True)

            # Finding the associated list and extracting its items
            list_element = subsection.find_next('div', class_='text parbase section')
            if list_element:
                list_items = list_element.find_all('li')
                biography[title] = [{'text': item.get_text(strip=True), 'link': item.find('a')['href'] if item.find('a') and ['href'] in item.find('a') else ''} for item in list_items]
            else:
                biography[title] = []

            faculty_info['biography'] = biography
    elif ('Res' in section.get_text(strip=True)):
        response = requests.get(section['href'])
        soup1 = BeautifulSoup(response.text, 'html.parser')
        # Initialize a dictionary to store the extracted data
        research_areas = {}
        # Find all subsections with class 'title section'
        subsections = soup1.find_all('div', class_='title section')
        list_items = {}
        idx = 0
        for subsection in subsections:
            # Extract the title from the h3 tag
            title_element = subsection.find('h3')
            if not title_element:
                continue

            title = title_element.get_text(strip=True)

            # Find the associated list and extract its items
            list_element = subsection.find_next('div', class_='list parbase section')
            if list_element:
                list_items["Research Area List Possibility "+str(idx)] = str(list_element.find_all('li'))
                idx+=1

        faculty_info['research_areas'] = list_items


    elif ('Teachi' in section.get_text(strip=True)):
        response = requests.get(section['href'])
        soup1 = BeautifulSoup(response.text, 'html.parser')

        # Assuming soup is already defined with your HTML content
        idx = 0
        list_elements = soup1.find_all('div', class_='text parbase section')
        for list_element in list_elements:
            if list_element:
                list_items = list_element.find_all('li')
                faculty_info["Course List Possibility "+str(idx)] = [{'text': item.get_text(strip=True).replace('\u2014', ' - '), 'link': item.find('a')['href'] if item.find('a') else ''} for item in list_items]
                idx+=1

  return faculty_info


# New section

In [7]:

# Replace 'path/to/yourfile.html' with the path to your saved HTML file
adjunct = '/content/adjunct.html'
affiliated = '/content/affiliated.html'
emeriti = '/content/emeriti.html'
fulltime = '/content/full-time.html'
inmem = '/content/inmem.html'

files = [{"name": "Adjunct", "path": adjunct},
        {"name": "Affiliated", "path": affiliated},
        {"name": "Emeriti", "path": emeriti},
        {"name": "Full-Time", "path": fulltime},
        {"name": "In-Memorial", "path": inmem}]

faculty_data = []
for file in files:
    # Open and read the file
    with open(file["path"], 'r') as f:
        html_content = f.read()

    # Use Beautiful Soup to parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Define a structure to hold the data

    # Find all <li> tags
    faculty_list_parent = soup.find_all('li')
    faculty_list = soup.find_all('li')

    # List to store faculty data


    for faculty in faculty_list:
        # Extracting details
        name_element = faculty.find('div', class_='profileinfo-teaser-name')

        if (not name_element):
          continue

        name = name_element.get_text(strip=True).split(',')[0] if name_element else "Name Not Available"
        page_link_element = faculty.find('a', class_='title')
        page_link = page_link_element['href'] if page_link_element else "Link Not Available"

        # qualification_element = faculty.find('small')
        # qualification = qualification_element.get_text(strip=True) if qualification_element else "Qualification Not Available"
        qualification_element = faculty.find('div', class_='profileinfo-teaser-degree')
        qualification = qualification_element.get_text(strip=True) if qualification_element else "Qualification Not Available"

        location_element = faculty.find('div', class_='profileinfo-teaser-contact')
        if location_element:
            p_elements = location_element.find_all('p')
            location = p_elements[0].get_text(strip=True) if len(p_elements) > 0 else "Location Not Available"
            telephone = p_elements[1].get_text(strip=True) if len(p_elements) > 1 else "Telephone Not Available"
        else:
            location = "Location Not Available"
            telephone = "Telephone Not Available"

        email_element = faculty.find('a', class_='longtext')
        email = email_element['href'] if email_element else "Email Not Available"

        dept_title_element = faculty.find('div', class_='profileinfo-teaser-dept-title')
        dept_title = dept_title_element.get_text(strip=True) if dept_title_element else "Dept Title Not Available"

        dept_name_element = faculty.find('div', class_='profileinfo-teaser-dept-name')
        dept_name = dept_name_element.get_text(strip=True) if dept_name_element else "Dept Name Not Available"

        school_name_element = faculty.find('div', class_='profileinfo-teaser-school-name')
        school_name = school_name_element.get_text(strip=True) if school_name_element else "School Name Not Available"

        research_interests_element = faculty.find('div', class_='profileinfo-teaser-interests')
        research_interests = research_interests_element.get_text(strip=True) if research_interests_element else "Research Interests Not Available"

        faculty_info = {}
        if (page_link_element):
          faculty_info = fetch_faculty_info(page_link)
        print(name)
        faculty_data.append({
            'type': file["name"],
            'name': name,
            'page_link': page_link,
            'qualification': qualification,
            'location': location,
            'telephone': telephone,
            'email': email,
            'dept_title': dept_title,
            'dept_name': dept_name,
            'school_name': school_name,
            'research_interests': research_interests,
            'faculty_info': faculty_info
        })


Mike Buckley
Souma Chowdhury
Adrienne Decker
Yingjie Hu
Kevin R. Keane
Kevin Kwiat
Zhen Liu
Marianthi Markatou
Maria Y. Rodriguez
Ryan St. Pierre
Matt Stock
Yijun Sun
Robert Sutor
Charles Tirone
Jun Xia
Chi Zhou
Jing Gao
Steve Ko
Dimitrios Koutsonikolas
Shi Li
Jingjing Meng
Chris Proctor
Lu Su
Weihang Wang
Laurence Boxer
Chang Wen Chen
Jan Chomicki
Daniel Fischer
Bharat Jayaraman
Helene Kershner
Anthony Ralston
William Rapaport
Peter Scott
Stuart C. Shapiro
Deborah Walters
Aidong Zhang
Nasrin Akhter
Carl Alphonce
Roshan Ayyalasomayajula
Ethan Blanton
Marina Blanton
Varun Chandola
Changyou Chen
Zongchen Chen
Karthik Dantu
Sreyasee Das Bhattacharjee
Murat Demirbas
Paul Dickson
David Doermann
Mingchen Gao
Venu Govindaraju
Jesse Hartloff
Tom Hayes
Xin (Roger) He
Matthew Hertz
Andrew Hirsch
Christopher Hoadley
Hongxin Hu
Alan Hunt
Kaiyi Ji
Zhanpeng Jin
Kenneth Joseph
Oliver Kennedy
Matthew Knepley
Tevfik Kosar
Vishnu Lokhande
Haonan Lu
Kelin Luo
Siwei Lyu
Eric Mikida
Russ Miller
Ifeoma Nwog

In [8]:
# Convert to JSON
json_data = json.dumps(faculty_data, indent=4)

# Print or save the JSON data
print(json_data)

[
    {
        "type": "Adjunct",
        "name": "Mike Buckley",
        "page_link": "/computer-science-engineering/people/faculty-directory/adjunct.host.html/content/shared/engineering/computer-science-engineering/profiles/faculty/adjunct/buckley-mike.detail.html",
        "qualification": "MS, Rochester Institute of Technology",
        "location": "356 Davis Hall",
        "telephone": "mikeb@buffalo.edu",
        "email": "mailto:mikeb@buffalo.edu",
        "dept_title": "Eminent Teaching Professor Emeritus",
        "dept_name": "Department of Computer Science and Engineering",
        "school_name": "School of Engineering and Applied Sciences",
        "research_interests": "Research Areas:Embedded systems; factory automation; software engineering; wireless telecommunications",
        "faculty_info": {
            "Course List Possibility 0": [
                {
                    "text": "CSE 442 - Software Engineering (Fall 2022)",
                    "link": ""
          

In [9]:
with open('faculty_data.json', 'w') as f:
    json.dump(faculty_data, f, indent=4)